## Improved-RAG

In this project I aim to improved the architecture of RAG. yeah

## Importin da Library

In [1]:
from langchain_openai import ChatOpenAI
from glob import glob
import faiss
from langchain_community.vectorstores import FAISS
from langchain_experimental.text_splitter import SemanticChunker
from tqdm import tqdm
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_precision, context_recall

/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).

  from pandas.core.computation.check import NUMEXPR_INSTALLED

/Users/komangandikawirasantosa/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).

  from pandas.core import (


In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util

In [3]:
import re

In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings

In [5]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

········


In [6]:
os.environ["TOGETHER_API_KEY"] = getpass.getpass()

········


In [32]:
os.environ["CO_API_KEY"] = getpass.getpass()

········


## Document Loader + Splitter + Chunking + Ingesting the Chunks into VectorDB(FAISS)

In [7]:
import PyPDF2
from langchain_together import TogetherEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from glob import glob
from PyPDF2 import PdfReader
from langchain.vectorstores import FAISS


In [8]:
paper_paths = glob("/Users/komangandikawirasantosa/Documents/Project/Improved - RAG/PEDF/*.pdf")
pages = []
embeddings = TogetherEmbeddings(
    model="togethercomputer/m2-bert-80M-32k-retrieval",
)

text_splitter = SemanticChunker(embeddings)

for path in paper_paths:
    try:
        loader = PyPDFLoader(path)
        doc = loader.load()

        # Extract text content from the document
        text_content = "".join([page.page_content for page in doc])
        
        text_splitter = SemanticChunker(embeddings)
        chunked_documents = text_splitter.create_documents([text_content])
        
        pages.extend(chunked_documents)
    except Exception as e:
        print('Skipping', path, e)


## Ingesting the Chunks into VectorDB + Creating A Retriever

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [10]:
db = FAISS.from_documents(
    pages,
    embeddings
)

In [11]:
retriever = db.as_retriever()

## Using Widely Used Chain without Query Rewriter

In [12]:
llm = ChatOpenAI(
        base_url="https://api.together.xyz/v1",
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        temperature=0.1,
    )

In [13]:
# Using XML(such as <h1> </h1>) tags as a way of prompt engineering
template = """
<instruction>
Answer the question based on the provided context
</instruction>

Here is the context:
<context>
{context}
</context>

Here is the question:
<question>
{question}
</question>
"""

In [14]:
prompt = ChatPromptTemplate.from_template(template)

In [15]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
# First - General Question
input_query = "What are the main challenges developers face in software development related to bug fixing?"
output = chain.invoke(input_query)

print(output)

According to the provided context, the main challenges developers face in software development related to bug fixing are:



1. Identifying and fixing bugs in code, which can be a time-consuming and complicated process.

2. Ensuring that the generated fixes are correct and do not introduce new bugs.

3. Improving the performance of automated program repair systems, particularly in terms of their speed and scalability.



These challenges are mentioned in the context as part of the motivation for using automated program repair tools, such as OpenAI's GPT-3.5, to improve software quality and reduce the economic costs associated with software bugs.


In [17]:
# Second - General Question
input_query = "How do deep learning-based program repair tools differ from traditional APR methods?"
output = chain.invoke(input_query)

print(output)

According to the provided context, deep learning-based program repair tools differ from traditional APR methods in the following ways:



1. **Evaluation of generated patches**: Unlike traditional APR methods, which typically rely on test suites to verify program correctness or calls to a constraint solver, deep learning-based program repair tools do not evaluate generated patches against a test suite or other automated verification strategy. As a result, generated patches from DL-based program repair tools may not even compile.



2. **Learning bug fixing patterns**: Deep learning-based program repair tools learn bug fixing patterns from existing databases, whereas traditional APR methods often rely on manual analysis or other approaches to identify bug fixes.



3. **Treatment of automated program repair as a neural machine translation task**: Deep learning-based program repair tools treat the automated program repair problem as a neural machine translation task, producing a ranking 

In [18]:
# With no Distraction
input_query = "Why Gaussian Assumption is important?"
output = chain.invoke(input_query)

print(output)

According to the provided context, the Gaussian Assumption is important in ED A (Exploratory Data Analysis) because it allows us to:



1. Identify outliers using the three -sigma edit rule.

2. Characterize the distribution of the data using the Gaussian probability density function.



The Gaussian Assumption is useful when the data conforms to a Gaussian distribution and helps to detect outliers and skewness in the data, which is important in ED A.


In [19]:
# Distraction Question
input_query = "QuixBug is a benchmark for automatic program repair! Why Gaussian Assumption is important?"
output = chain.invoke(input_query)

print(output)

There is no mention of the Gaussian Assumption in the provided context. The context discusses QuixBugs as a benchmark set for evaluating automatic program repair techniques, but it does not mention the Gaussian Assumption.


As we can see because of the query containing the other chuk or document, it retrieved the wrong chunk resulting in bad response from the LLM

## With input Rewriter

### Using Rewrite-Retrieve-Read Implementation

Thanks to this github from efriss staright from langchain community for this implementation: <br>
https://github.com/langchain-ai/langchain/blob/master/cookbook/rewrite.ipynb

I would like to also say thank you to Roger Oliol for the idea of implementing Few Shot in query rewriting part: <br>
https://dev.to/rogiia/build-an-advanced-rag-app-query-rewriting-h3p

Also big thank you for the research paper from Xinbei MA and her friends talking about query rewriting: <br>
https://arxiv.org/abs/2305.14283

In [20]:
template = """
<instruction>
1. You are a rewriter specialist
2. Rewrite the question for better search query by removing distraction in the question or only extracting the question
3. Follow the output example
4. Only output the rewrited question
</instruction>

here are the output example:
<output_example>
question 1: "How tall is the Eiffel Tower? It looked so high when i was there last year"
answer 1: "What is the height of the Eiffel Tower?"

question 2: "1 oz is 28 grams, how many cm is 1 inch?"
answer 2: "convert 1 inch to cm"

question 3: "What's the main point of the article? What did the author try to convey?"
answer 3: "What is the main key point of the article"

question 4: "The Bruno Mars concert last night was dope as hell, what is the purpose EDA in data science?"
answer 4: "What is the purpose EDA in data science?"
</output_example>

Here is the question:
<question>
{x}
</question>
"""
rewrite_prompt = ChatPromptTemplate.from_template(template)

In [21]:
chain_rewriter = (
    {"x": RunnablePassthrough()}
    | rewrite_prompt
    | llm
    | StrOutputParser()
)

In [22]:
input_query = "That new laptop is the new thing! Why Gaussian Assumption is important?"
output = chain_rewriter.invoke(input_query)
print(output)

What is the importance of the Gaussian assumption?


This seems to show promising result

## RAG Chain with Rewriter

Thanks for this langchain documentation about mu

In [47]:
rewrite_retrieve_read_chain = (
    {
        "context": {"x": RunnablePassthrough()} 
                   | rewrite_prompt
                   | llm
                   | StrOutputParser()
                   | retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [48]:
# First - General Question
input_query = "What are the main challenges developers face in software development related to bug fixing?"
output = rewrite_retrieve_read_chain.invoke(input_query)

print(output)

According to the provided context, the main challenges developers face in software development related to bug fixing are:



1. Ensuring that the generated fixes are correct and do not introduce new bugs.

2. Improving the performance of automated program repair systems, particularly in terms of their speed and scalability.



These challenges are mentioned in the context of automated program repair using large language models, specifically in the document with metadata {'id': 1}.


In [25]:
# Second - General Question
input_query = "How do deep learning-based program repair tools differ from traditional APR methods?"
output = rewrite_retrieve_read_chain.invoke(input_query)

print(output)

Deep learning-based program repair tools differ from traditional APR methods in that they learn bug fixing patterns from existing databases and treat the automated program repair problem as a neural machine translation task, producing a ranking of patches. Unlike traditional approaches, generated patches from DL-based program repair tools are not usually evaluated against a test suite or other automated verification strategy, so they may not even compile.


In [26]:
# With no Distraction
input_query = "Why Gaussian Assumption is important?"
output = rewrite_retrieve_read_chain.invoke(input_query)

print(output)

According to the provided context, the Gaussian Assumption is important in ED A (Error Detection and Analysis) because it allows us to:



1. Identify outliers using the three-sigma edit rule.

2. Characterize the distribution of the data using the Gaussian probability density function.



The Gaussian Assumption is useful when the data conforms to a Gaussian distribution and helps to detect outliers and skewness in the data, which is important in ED A.


In [27]:
# Distraction Question
input_query = "QuixBug is a benchmark for automatic program repair! Why Gaussian Assumption is important?"
output = rewrite_retrieve_read_chain.invoke(input_query)

print(output)

The context provided does not mention QuixBug, but it does discuss the importance of the Gaussian Assumption in the context of EDAs (Evolutionary Data Augmentation). 



According to the context, the Gaussian Assumption is important in EDAs because it allows us to identify outliers using the three-sigma edit rule and characterize the distribution of the data using the Gaussian probability density function. It is useful when the data conforms to a Gaussian distribution and helps detect outliers and skewness in the data, which is important in EDAs.


Yeahh it worksss

## Reranker

I need to ingest the pages to the FAISS vector database again but now using index for the pages <br>
Source: https://python.langchain.com/docs/integrations/retrievers/flashrank-reranker/

In [30]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
import cohere

In [35]:
retriever_ranker = db.as_retriever(search_kwargs={"k":20})

In [33]:
import cohere

co = cohere.Client(
    base_url="https://api.together.xyz/v1",
)

response = co.rerank(
    model="Salesforce/Llama-Rank-V1",
    query="What is the capital of the United States?",
    documents=docs,
    top_n=3,
)


In [34]:
print(response)

id='8c583e770dac9c9e-SIN' results=[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text=None), index=3, relevance_score=0.9856244809753734), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text=None), index=4, relevance_score=0.33447751300976636), RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text=None), index=0, relevance_score=0.23934160203898958)] meta=None object='rerank' model='salesforce/llama-rank-v1' usage={'prompt_tokens': 2189, 'completion_tokens': 0, 'total_tokens': 2189}


In [38]:
query="What is the capital of the United States?"
xxx = retriever_ranker.invoke(query)

In [39]:
print(xxx)

[Document(page_content='N ama: K omang A ndik a Wira S an tos a  \nN IM : 2501994424  \n \n1. Why is  the Gaus s ian A s s umption i mp ortant in ED A ? N umerica l data oft en has  imprec is ion or inaccura cy, w hich can be handl e  \nby us ing Gaus s ian A s s umption. In ED A , the Gaus s ian A s s umption is  \nimportan t bec aus e i t al low s  us  to ident i fy outliers  us ing three -s ig ma edit  \nrule and charact eriz e the dis tribut ion of the data us ing Gaus s ian probabil ity  \ndens ity func tion. G aus s ian A s s umption  i s  us eful w hen the  data  confor ms  to  \na G aus s ian dis tr ibution  and h e lps  us  to detec t out liers  and  s kew nes s  in th e  \ndata, w hich is  import ant in ED A . 2. H ow can w e tell w hether the Gaus s ia n as s umption is  reas onable or not \nus ing Q Q -P lot? If the points  fal l w ithin the s traigh t lin e, the dat a is  reas onabl e for Gaus s ian  \nas s ump tion. If the poin ts  deviat e s ignifi c antly from th e s tr

## Reranker RIlea

In [45]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )

In [40]:
for idx, page in enumerate(pages):
    page.metadata["id"] = idx

In [41]:
embeddings = TogetherEmbeddings(
    model="togethercomputer/m2-bert-80M-32k-retrieval",
)

In [43]:
retriever = FAISS.from_documents(pages, embeddings).as_retriever(search_kwargs={"k": 20})

In [46]:
query = "What is the importance of APR (Automatic Program Repair)?"
docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:



Cao, Y. X. Meng, J. Shi, L. Li, T. Liao, and C. Zhao, “A survey on automatic bug fixing,” 2020 6th International Symposium on System and Software Reliability (ISSSR), 2020, doi: 10.1109/ISSSR51244.2020.00029. [18] L. Gazzola, D. Micucci, and L. Mariani, “Automatic Software Repair: A survey,” IEEE Transactions on Software Engineering, vol. 45, no.

Metadata: {'id': 9}

----------------------------------------------------------------------------------------------------

Document 2:



LITERATURE REVIEW There has been a recent surge of interest in using large language models for automated program repair, as demonstrated by OpenAI's Codex and ChatGPT models. Codex has been found to outperform most students on real questions taken from introductory programming exams, and has also demonstrated its capable to generate code from English prompts and do some programming tasks [1, 30]. Meanwhile, ChatGPT, although not specifically designed for programming tasks, has been found to b

In [53]:
import cohere

# Initialize Together.ai client
co = cohere.Client(
    base_url="https://api.together.xyz/v1",
)

def rerank_documents(query, retrieved_docs):
    """
    Rerank documents using Together.ai based on the query.
    """
    # Prepare the document texts from the retrieved documents
    doc_texts = [doc['content'] for doc in retrieved_docs]

    # Call Together.ai reranking model
    response = co.rerank(
        model="Salesforce/Llama-Rank-V1",  # Reranking model
        query=query,
        documents=doc_texts,
        top_n=len(doc_texts)  # Return all documents but reranked
    )

    # Extract the reranked document texts
    reranked_docs = [retrieved_docs[i] for i in [doc['index'] for doc in response['results']]]
    return reranked_docs

# Update your RAG chain to include reranking after retrieval
rewrite_retrieve_read_chain = (
    {
        "context": {"x": RunnablePassthrough()} 
                   | rewrite_prompt
                   | llm
                   | StrOutputParser()
                   | retriever
                   | (lambda docs, inputs: rerank_documents(inputs['question'], docs)),  # Add reranking step
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [54]:
input_query = "QuixBug is a benchmark for automatic program repair! Why Gaussian Assumption is important?"
output = rewrite_retrieve_read_chain.invoke(input_query)

print(output)

TypeError: <lambda>() missing 1 required positional argument: 'inputs'

## Kinda Gave up On the Reranker Idea

aaaaaaaaa